# Generate Data for Analysis of Iris Visibility

This notebook generates data for analysis of glint visibility.
Generation of 100 combined user-device samples x two eyes x 20 gaze directions,
i.e., 4,000 individual samples, takes about 10-15 min on a beefy machine.

In [1]:
%matplotlib widget

import datetime
import kiruna
import os
import pickle
import sys
import torch

sys.path.append("..")
import utils

dropdown, text = utils.get_experiment_info()

RadioButtons(description='Device:', index=4, options=('atlas_1.2', 'palau', 'sydney', 'tasman', 'p47', 'p47_PO…

Text(value='.\\results\\p47', description='Results:', placeholder="Default is '.'")

## Data Generation

This takes 10-15 min on a beefy machine.

In [2]:
scene_file_name, \
    sampler_file_name = \
    utils.get_configuration_files(dropdown.value)

print("Scene generated using " + scene_file_name + " configuration file.")
et_scene = \
    kiruna.scene.SceneModel(
        parameter_file_name=scene_file_name, requires_grad=True
    )

print("Sampling parameters from " + sampler_file_name + " configuration file.")
scene_sampler = kiruna.sampler.SceneSampler(
    et_scene, num_samples=100, parameter_file_name=sampler_file_name)

df = scene_sampler.generate_data_for_iris_analysis()

now = datetime.datetime.now()
prefix = "iris - " + now.strftime("%Y-%m-%d @ %H-%M-%S.%f")
results_path = text.value
os.makedirs(results_path, exist_ok=True)
path_prefix = os.path.join(results_path, prefix)

df_name = path_prefix + " data_frame.pkl"
with open(df_name, 'wb') as file_stream:
    pickle.dump(df, file_stream)

df

Scene generated using C:\Users\jayanez\Documents\mixedreality.Platform.EyeTracking.Tools\kiruna\kiruna\scene\p47_POC_scene/p47_POC_scene.json configuration file.
Sampling parameters from C:\Users\jayanez\Documents\mixedreality.Platform.EyeTracking.Tools\kiruna\kiruna\sampler\default_sampler/default_scene_sampler.json configuration file.


c:\Users\jayanez\Documents\mixedreality.Platform.EyeTracking.Tools\kiruna\envs\KIRUNA_env\Lib\site-packages\torch\autograd\__init__.py:380: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11070). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at C:\cb\pytorch_1000000000000\work\c10\cuda\CUDAFunctions.cpp:108.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


,Subsystem,Percentage visible,Area in iris [mm^2],Area in image [pix^2]
0,0,100.000008,101.787613,7316.874512
1,1,100.000008,101.787613,7286.141113
2,0,100.000008,101.787613,5841.924316
3,1,100.000008,101.787613,5952.185059
4,0,100.000008,101.787613,4911.940918
...,...,...,...,...
195,1,100.000008,101.787613,5140.835938
196,0,100.000008,101.787613,5297.475098
197,1,100.000008,101.787613,5796.025879
198,0,100.000008,101.787613,5624.375000
